# 1. Loading data

In [40]:
import pandas as pd

In [41]:
def load_data(split_name='train', columns=['text', 'label'], folder='data'):
    '''
        "split_name" may be set as 'train', 'valid' or 'test' to load the corresponding dataset.
        
        You may also specify the column names to load any columns in the .csv data file.
        Among many, "text" can be used as model input, and "label" column is the labels (sentiment). 
    '''
    try:
        print(f"select [{', '.join(columns)}] columns from the {split_name} split")
        df = pd.read_csv(f'{folder}/{split_name}.csv')
        df = df.loc[:,columns]
        print("Success")
        return df
    except:
        print(f"Failed loading specified columns... Returning all columns from the {split_name} split")
        df = pd.read_csv(f'{folder}/{split_name}.csv')
        return df

In [42]:
train_df = load_data('train', columns=['text', 'label'], folder='data')
valid_df = load_data('valid', columns=['text', 'label'], folder='data')
# the test set labels (the 'label' column) are unavailable! So the following code will instead return all columns
test_df = load_data('test_no_label', columns=['id', 'text'], folder='data')

select [text, label] columns from the train split
Success
select [text, label] columns from the valid split
Success
select [id, text] columns from the test_no_label split
Success


# 2. Data preprocessing functions

In [43]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stopwords = set (stopwords.words('english'))
porterStemmer = PorterStemmer()

def lower(s):
    """
    :param s: a string.
    return a string with lower characters
    Note that we allow the input to be nested string of a list.
    e.g.
    Input: 'Text mining is to identify useful information.'
    Output: 'text mining is to identify useful information.'
    """
    if isinstance(s, list):
        return [lower(t) for t in s]
    if isinstance(s, str):
        return s.lower()
    else:
        raise NotImplementedError("unknown datatype")


def tokenize(text):
    """
    :param text: a doc with multiple sentences, type: str
    return a word list, type: list
    e.g.
    Input: 'Text mining is to identify useful information.'
    Output: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    """
    return nltk.word_tokenize(text)


def stem(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of stemmed words, type: list
    e.g.
    Input: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    Output: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     results.append(ps.stem(token))
    # return results

    return [porterStemmer.stem(token) for token in tokens]

def n_gram(tokens, n=1):
    """
    :param tokens: a list of tokens, type: list
    :param n: the corresponding n-gram, type: int
    return a list of n-gram tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.'], 2
    Output: ['text mine', 'mine is', 'is to', 'to identifi', 'identifi use', 'use inform', 'inform .']
    """
    if n == 1:
        return tokens
    else:
        results = list()
        for i in range(len(tokens)-n+1):
            # tokens[i:i+n] will return a sublist from i th to i+n th (i+n th is not included)
            results.append(" ".join(tokens[i:i+n]))
        return results

def filter_stopwords(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of filtered tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    Output: ['text', 'mine', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     if token not in stopwords and not token.isnumeric():
    #         results.append(token)
    # return results

    return [token for token in tokens if token not in stopwords and not token.isnumeric()]


def get_onehot_vector(feats, feats_dict):
    """
    :param data: a list of features, type: list
    :param feats_dict: a dict from features to indices, type: dict
    return a feature vector,
    """
    # initialize the vector as all zeros
    vector = np.zeros(len(feats_dict), dtype=float)
    for f in feats:
        # get the feature index, return -1 if the feature is not existed
        f_idx = feats_dict.get(f, -1)
        if f_idx != -1:
            # set the corresponding element as 1
            vector[f_idx] = 1
    return vector

def biGram(tokens):
    return n_gram(tokens, 2)

# 3. Build our model

## A. CountVectorizer + Logistic regression

### Import library

In [44]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

### Data preprocessing

In [45]:
train_df = load_data('train')
valid_df = load_data('valid')
x_train = train_df['text']
y_train = train_df['label']
x_valid = valid_df['text']
y_valid = valid_df['label']

select [text, label] columns from the train split
Success
select [text, label] columns from the valid split
Success


In [46]:
train_data_x = train_df['text'].map(tokenize).map(lower).map(filter_stopwords).map(stem)
train_data_y = train_df['label']
valid_data_x = valid_df['text'].map(tokenize).map(lower).map(filter_stopwords).map(stem)
valid_data_y = valid_df['label']

In [47]:
for i in range(len(train_data_x)):
    train_data_x[i] = ' '.join(train_data_x[i])
for i in range(len(valid_data_x)):
    valid_data_x[i] = ' '.join(valid_data_x[i])

In [48]:
train_data_x.to_csv("data/norm_train_data.csv")
valid_data_x.to_csv("data/norm_valid_data.csv")

In [49]:
norm_train_data = load_data("norm_train_data")['text']
norm_valid_data = load_data("norm_valid_data")['text']

select [text, label] columns from the norm_train_data split
Failed loading specified columns... Returning all columns from the norm_train_data split
select [text, label] columns from the norm_valid_data split
Failed loading specified columns... Returning all columns from the norm_valid_data split


### Model

In [50]:
countVectorizer = CountVectorizer(min_df=0.0, max_df=0.2, binary=False, ngram_range=(1, 3))
cV_train = countVectorizer.fit_transform(norm_train_data)
cV_valid = countVectorizer.transform(norm_valid_data)

In [51]:
lr = LogisticRegression()
lr.fit(cV_train, y_train)
lr

LogisticRegression()

In [52]:
y_pred = lr.predict(cV_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

              precision    recall  f1-score   support

           1       0.57      0.52      0.54       295
           2       0.38      0.19      0.25       198
           3       0.47      0.55      0.51       508
           4       0.50      0.46      0.47       523
           5       0.61      0.71      0.65       476

    accuracy                           0.52      2000
   macro avg       0.50      0.48      0.49      2000
weighted avg       0.51      0.52      0.51      2000




[[153  28  73  20  21]
 [ 47  38  82  21  10]
 [ 41  27 279 116  45]
 [ 15   5 122 238 143]
 [ 11   3  39  85 338]]
accuracy 0.523


## B. LSTM

### Import library

In [67]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import tqdm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from torch.autograd import Variable

In [68]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")

### Data preprocessing

In [93]:
train_df = load_data('train')
valid_df = load_data('valid')
x_train = train_df['text'].map(tokenize).map(lower).map(filter_stopwords).map(stem)
y_train = train_df['label']
x_valid = valid_df['text'].map(tokenize).map(lower).map(filter_stopwords).map(stem)
y_valid = valid_df['label']

select [text, label] columns from the train split
Success
select [text, label] columns from the valid split
Success


In [94]:
for i in range(len(x_train)):
    x_train[i] = ' '.join(x_train[i])
for i in range(len(x_valid)):
    x_valid[i] = ' '.join(x_valid[i])
x_train.to_csv("data/norm_train_data.csv")
x_valid.to_csv("data/norm_valid_data.csv")

In [95]:
norm_train_data = load_data("norm_train_data")['text']
norm_valid_data = load_data("norm_valid_data")['text']

select [text, label] columns from the norm_train_data split
Failed loading specified columns... Returning all columns from the norm_train_data split
select [text, label] columns from the norm_valid_data split
Failed loading specified columns... Returning all columns from the norm_valid_data split


In [96]:
word2id = {}
for tokens in norm_train_data:
    for t in tokens:
        if not t in word2id:
            word2id[t] = len(word2id)
word2id['<pad>'] = len(word2id)

In [97]:
def texts_to_id_seq(texts, padding_length=50):
    records = []
    for tokens in texts:
        record = []
        for t in tokens:
            record.append(word2id.get(t, len(word2id)))
        if len(record) >= padding_length:
            records.append(record[:padding_length])
        else:
            records.append(record + [word2id['<pad>']] * (padding_length - len(record)))
    return records

In [98]:
train_seqs = texts_to_id_seq(norm_train_data)
valid_seqs = texts_to_id_seq(norm_valid_data)

In [99]:
X_train_tensors = Variable(torch.Tensor(train_seqs)).type(torch.LongTensor)
X_valid_tensors = Variable(torch.Tensor(valid_seqs)).type(torch.LongTensor)

y_train_tensors = Variable(torch.Tensor(y_train)).type(torch.LongTensor)
y_valid_tensors = Variable(torch.Tensor(y_valid)).type(torch.LongTensor)

In [100]:
class MyDataset(Dataset):
    
    def __init__(self, seq, y):
        assert len(seq) == len(y)
        self.seq = seq
        self.y = y-1
    
    def __getitem__(self, idx):
        return np.asarray(self.seq[idx]), self.y[idx]

    def __len__(self):
        return len(self.seq)

In [101]:
batch_size = 16

train_loader = DataLoader(MyDataset(X_train_tensors, y_train_tensors), batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(MyDataset(X_valid_tensors, y_valid_tensors), batch_size=batch_size)

### Model

In [102]:
class lstm_model(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(lstm_model, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.embedding = nn.Embedding(num_embeddings=len(word2id)+1, embedding_dim=input_size)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.max = nn.MaxPool1d(kernel_size=3,
                                stride=1)
        self.drop = nn.Dropout(0.5)
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        embedd = self.embedding(x)
        embedd = self.drop(embedd)
        output, (hn, cn) = self.lstm(embedd, (h_0, c_0)) #lstm with input, hidden, and internal state
        # return self.softmax(self.fc(output[:,-1,:]))
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        # out = self.fc_1(out) #first Dense
        # out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [103]:
num_classes, input_size, hidden_size, num_layers, seq_length = 5, 64, 128, 1, 1
model = lstm_model(num_classes, input_size, hidden_size, num_layers, seq_length)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()     

In [104]:
for e in range(1, 31):    
    print('epoch', e)
    model.train()
    total_acc = 0
    total_loss = 0
    total_count = 0
    with tqdm.tqdm(train_loader) as t:
        for x, y in t:
            # print(x.shape)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            total_acc += (logits.argmax(1) == y).sum().item()
            total_count += y.size(0)
            total_loss += loss.item()
            optimizer.step()
            t.set_postfix({'loss': total_loss/total_count, 'acc': total_acc/total_count})

    model.eval()
    y_pred = []
    y_true = []
    with tqdm.tqdm(valid_loader) as t:
        for x, y in t:
            logits = model(x)
            total_acc += (logits.argmax(1) == y).sum().item()
            total_count += len(y)
            y_pred += logits.argmax(1).tolist()
            y_true += y.tolist()
    print(classification_report(y_true, y_pred))
    print("\n\n")
    print(confusion_matrix(y_true, y_pred))
    print('accuracy', np.mean(y_valid == y_pred))

epoch 1


100%|██████████| 125/125 [00:01<00:00, 116.15it/s]
/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       295
           1       0.00      0.00      0.00       198
           2       0.00      0.00      0.00       508
           3       0.28      0.27      0.27       523
           4       0.24      0.77      0.37       476

    accuracy                           0.25      2000
   macro avg       0.10      0.21      0.13      2000
weighted avg       0.13      0.25      0.16      2000




[[  0   0   0  80 215]
 [  0   0   0  47 151]
 [  0   0   0 125 383]
 [  0   0   0 139 384]
 [  0   0   0 110 366]]
accuracy 0.2545
epoch 2


100%|██████████| 125/125 [00:01<00:00, 103.83it/s]
/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/him/anaconda3/envs/COMP4332_Project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       295
           1       0.00      0.00      0.00       198
           2       0.26      0.37      0.30       508
           3       0.29      0.34      0.32       523
           4       0.29      0.39      0.33       476

    accuracy                           0.28      2000
   macro avg       0.17      0.22      0.19      2000
weighted avg       0.21      0.28      0.24      2000




[[  0   0 122  92  81]
 [  0   0  93  59  46]
 [  0   0 189 158 161]
 [  0   0 174 180 169]
 [  0   0 161 128 187]]
accuracy 0.21
epoch 3


...

              precision    recall  f1-score   support

           0       0.46      0.39      0.42       295
           1       0.30      0.11      0.16       198
           2       0.37      0.49      0.42       508
           3       0.37      0.29      0.33       523
           4       0.49      0.62      0.55       476

    accuracy                           0.42      2000
   macro avg       0.40      0.38      0.38      2000
weighted avg       0.41      0.42      0.40      2000




[[114  19 104  21  37]
 [ 53  22  80  21  22]
 [ 46  19 248 119  76]
 [ 20   9 163 151 180]
 [ 14   4  69  92 297]]
accuracy 0.199
epoch 29


100%|██████████| 125/125 [00:01<00:00, 109.75it/s]


              precision    recall  f1-score   support

           0       0.44      0.44      0.44       295
           1       0.29      0.12      0.17       198
           2       0.38      0.53      0.44       508
           3       0.40      0.27      0.33       523
           4       0.51      0.60      0.55       476

    accuracy                           0.42      2000
   macro avg       0.40      0.39      0.38      2000
weighted avg       0.42      0.42      0.41      2000




[[131  22  97  20  25]
 [ 58  23  79  18  20]
 [ 67  16 268  86  71]
 [ 28  11 182 143 159]
 [ 17   6  79  89 285]]
accuracy 0.173
epoch 30


100%|██████████| 125/125 [00:01<00:00, 108.77it/s]

              precision    recall  f1-score   support

           0       0.47      0.35      0.40       295
           1       0.33      0.10      0.16       198
           2       0.38      0.47      0.42       508
           3       0.37      0.35      0.36       523
           4       0.49      0.61      0.55       476

    accuracy                           0.42      2000
   macro avg       0.41      0.38      0.38      2000
weighted avg       0.41      0.42      0.41      2000




[[104  20 101  32  38]
 [ 48  20  77  29  24]
 [ 41  11 240 140  76]
 [ 16   6 154 185 162]
 [ 12   3  59 110 292]]
accuracy 0.1995
